# Project for Algorithms in Molecular Biology

In [198]:
import numpy as np
import random
import math
from random import randint    

In [263]:
IUPAC = {
  'W' : ["A", "T"],
  "R" : ["A", "G"],
  "K" : ["G", "T"],
  "S" : ["G", "C"],
  "Y" : ["T", "C"],
  "M" : ["A", "C"],
  "B" : ["G", "C", "T"],
  "D" : ["G", "A", "T"],
  "H" : ["C", "A", "T"],
  "V" : ["C", "A", "G"],
  "N" : ["C", "A", "G", "T"],
}

IUPAC_bases = {
  "A": ["W", "R", "M", "D", "H", "V", "N"],
  "C": ["S", "Y", "M", "B", "H", "V", "N"],
  "G": ["R", "K", "S", "B", "D", "V", "N"],
  "T": ["W", "K", "Y", "B", "D", "H", "N"],
}

b_k = 0.25

In [200]:
# convert string to list character-wise
def ConvertStringToList(string):
  list1=[]
  list1[:0]=string
  return list1

In [201]:
def hammingDistance(str1, str2):
  diffs = 0
  for ch1, ch2 in zip(str1, str2):
    if ch1 != ch2:
      diffs += 1
  return diffs

In [202]:
def BuildFrequencies(motifs):
	k = len(motifs[0])
	frequencies = [[0 for y in range(k)] for x in range(4)]
	for count in range(k):
		A=0
		C=0
		G=0
		T=0
		
		# Add in Laplace counts to avoid
		# prob densities that are zero or one
		# accelerates alg runtime
		# A += 1
		# C += 1
		# G += 1
		# T += 1
		for string in motifs:
			if string[count]=='A':
				A+=1
			elif string[count]=='C':
				C+=1
			elif string[count]=='G':
				G+=1
			elif string[count]=='T':
				T+=1
		# Insert frequencies if base A
		frequencies[0][count] = (float(A) + b_k)/(A+C+G+T+1)
		# Insert frequencies if base C
		frequencies[1][count] = (float(C) + b_k)/(A+C+G+T+1)
		# Insert frequencies if base G
		frequencies[2][count] = (float(G)+ b_k)/(A+C+G+T+1)
		# Insert frequencies if base T
		frequencies[3][count] = (float(T) + b_k)/(A+C+G+T+1)
	return frequencies

In [203]:
def BuildPWM(frequencies):
  pwm = [[0 for y in range(np.shape(frequencies)[1])] for x in range(4)]
  for index1, prob in enumerate(frequencies):
    for index2, j in enumerate(prob):
       pwm[index1][index2] = math.log2(j/b_k)
  
  return pwm

In [204]:
# Evalation function that calculate the similarity between pattern string and PWM
def motif_evaluation(motif, frequencies_matrix, pwm):
  sum = 0
  for index, string in enumerate(motif):
    if string=='A':
      sum += frequencies_matrix[0][index]*pwm[0][index]
    elif string=='C':
      sum += frequencies_matrix[1][index]*pwm[1][index]
    elif string=='G':
      sum += frequencies_matrix[2][index]*pwm[2][index]
    elif string=='T':
      sum += frequencies_matrix[3][index]*pwm[3][index]
      
  return sum

In [205]:
# Scoring function of possible motifs
def scoring(motif, pwm):
  sum = 0
  for index, string in enumerate(motif):
    if string=='A':
      sum += pwm[0][index]
    elif string=='C':
      sum += pwm[1][index]
    elif string=='G':
      sum += pwm[2][index]
    elif string=='T':
      sum += pwm[3][index]
  return sum

In [206]:
def score(motifs):
	k = len(motifs[0])
	pattern = []
	for i in range(k):
		A=0
		C=0
		G=0
		T=0
		for string in motifs:
			if string[i]=='A':
				A+=1
			elif string[i]=='C':
				C+=1
			elif string[i]=='G':
				G+=1
			elif string[i]=='T':
				T+=1				
		if A >= C and A >= G and A >= T:
			pattern.append('A')
		elif C >= G and C >= T:
			pattern.append('C')
		elif G >= T:
			pattern.append('G')
		else:
			pattern.append('T')

	pattern = "".join(pattern)
 			
	score = 0
	for string in motifs:
		score += hammingDistance(string, pattern)
	return score

In [214]:
def next_population(motifs, dna, pwm):
  k = len(motifs[0])

  newMotifs = []
  # Calculate probilities for each k-mer in Dna_i
  for dna_string in dna:
    scores = [0 for x in range(len(dna_string)-k+1)]
    
    for i in range(len(dna_string)-k+1):
      # calculate the distances of each k-mer with
      scores[i] = scoring(dna_string[i:i+k], pwm)

    # set the threshold for selection
    threshold = np.mean(scores)

    for i in range(len(dna_string)-k+1):
      if scores[i] > threshold: # the higher the score, the bigger the chance to be selected
        newMotifs.append(dna_string[i:i+k])

  return newMotifs   

In [262]:
def selection(motifs_next_population, frequencies_matrix, pwm, len_of_motifs):
  selected_motifs = []
  scores = {}

  for motif in motifs_next_population: 
    scores[motif] = motif_evaluation(motif, frequencies_matrix, pwm)
  
  # select 2m of next population with the higher scores
  for _ in range(len_of_motifs*2):
    selected_motifs.append(max(scores, key=scores.get))
    del scores[max(scores, key=scores.get)]
    
  return selected_motifs

In [243]:
def finalSelection(dna, k, pwm, frequencies):
  motifs = []

  # Calculate probilities for each k-mer in Dna_i
  for index, dna_string in enumerate(dna):
    scoring = [0 for x in range(len(dna_string)-k+1)]
    
    for i in range(len(dna_string)-k+1):
      # calculate the score of each k-mer
      scoring[i] = motif_evaluation(dna_string[i:i+k], frequencies, pwm)

    # get the index of motif with the higher score
    max_index = scoring.index(max(scoring))
    motifs.append(dna_string[max_index:max_index+k])

  return motifs
  

In [210]:
# crossover two parents to create two children
def crossover(p1, p2):
	# children are copies of parents by default
  # c1, c2 = p1.copy(), p2.copy()
  # select crossover point that is not on the end of the string
  pt = randint(1, len(p1)-2)
  # perform crossover
  c1 = p1[:pt] + p2[pt:]
  c2 = p2[:pt] + p1[pt:]

  return [c1, c2]

In [278]:
def mutation(child):
  child_arr = ConvertStringToList(child)
  # get a random position to replace a base
  ind = randint(0, len(child)-1)

  # get a random position to get a char from IUPAC
  i = randint(0, len(IUPAC_bases[child_arr[ind]]) - 1)
  child_arr[ind] = IUPAC_bases[child[ind]][i]

  s = ""
  return s.join(child_arr) 

In [281]:
def GSGA(dna, k, N):
  motifs = []
  t = len(dna)

  for strand in dna:
    i = random.randrange(len(strand)-k+1)
    substr = strand[i:i+k]
    motifs.append(substr)

  # best_motifs = motifs
  # best_motifs_score = score(best_motifs)
  frequencies_matrix = BuildFrequencies(motifs)
  pwm = BuildPWM(frequencies_matrix)

  for _ in range(1,N):
    # motifsSelection = np.concatenate((best_motifs, selection(best_motifs, dna, frequences_matrix, pwm)))
    motifs_next_population = next_population(motifs, dna, pwm)
    motifs_selection = selection(motifs_next_population, frequencies_matrix, pwm, t)

    # create the next generation
    children = []
    for i in range(0, len(motifs_selection), 2):
      # get selected parents with mutation in pairs 
      p1, p2 = mutation(motifs_selection[i]), mutation(motifs_selection[i+1])
      child_score = {}
      # crossover
      for c in crossover(p1, p2):
        # evaluate the motif and add score to dictionary
        child_score[c] = motif_evaluation(c, frequencies_matrix, pwm)
      
      # add the individual with higher score
      children.append(max(child_score, key=child_score.get))

    # build the new position weight matrix based on the new population
    motifs = children
    frequencies_matrix = BuildFrequencies(children)
    pwm = BuildPWM(frequencies_matrix)

    # if score(children) < best_motifs_score:
    #   best_motifs = children
    #   best_motifs_score = score(children)

  final_motifs = finalSelection(dna, k, pwm, frequencies_matrix)

  return [final_motifs, score(final_motifs)]

In [286]:
kmer_length, N = 8, 2
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA', 'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG', 'TAGTACCGAGACCGAAAGAAGTATACAGGCGT', 'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC','AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
best_motifs = [None, float('inf')]

motifs = GSGA(dna, kmer_length, N)
print(motifs)

[['CAGTAACC', 'ATGTGTAA', 'AAGTATAC', 'AAGTTTCA', 'CAATGTTG'], 14]


- TCTCGGGG
- CCAAGGTG
- TACAGGCG
- TTCAGGTG
- TCCACGTG